# Urban Water Model Simulation for Fehraltorf

This notebook runs the urban water model simulation for Fehraltorf and visualizes the results.

In [ ]:
from pathlib import Path
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import folium
import geopandas as gpd
from branca.colormap import LinearColormap

from duwcm import run
from duwcm.functions import load_config

## Configuration

Set up the configuration for the simulation.

In [ ]:
# Load configuration
config_path = 'config.toml'
config = load_config(config_path, env='default')

# Display some key configuration settings
print(f"Input directory: {config.input_directory}")
print(f"Output directory: {config.output.output_directory}")
print(f"Simulation period: {config.simulation.start_year} - {config.simulation.end_year}")

## Run Simulation

Execute the urban water model simulation.

In [ ]:
# Run the simulation
results, model_params, forcing_data, flow_paths = run(config)

## Visualize Results

Create plots to visualize the simulation results.

In [ ]:
def plot_results(aggregated_results, forcing):
    plot_configs = [
        'Stormwater',
        'Baseflow',
        'Wastewater',
        'Evapotranspiration'
    ]
    
    total_area = aggregated_results.attrs['total_area']
    units = aggregated_results.attrs['units']
    
    plot_data = pd.DataFrame({
        'Precipitation': forcing['precipitation'],
        'Evaporation': forcing['potential_evaporation'],
        'Evapotranspiration': (aggregated_results['evaporation'] + aggregated_results['transpiration']) / total_area,
        'Stormwater': aggregated_results['stormwater'] / total_area,
        'Wastewater': aggregated_results['wastewater'] / total_area,
        'Baseflow': aggregated_results['baseflow'] / total_area
    })
    
    fig, axes = plt.subplots(2, 2, figsize=(20, 16))
    axes = axes.flatten()
    
    for i, config in enumerate(plot_configs):
        ax1 = axes[i]
        ax2 = ax1.twinx()
        
        ax1.fill_between(plot_data.index, 0, plot_data['Precipitation'],
                         facecolor='C0', alpha=0.3, label='Precipitation')
        ax1.plot(plot_data.index, plot_data['Evaporation'], linestyle='--', color='C1', label='Evaporation')
        ax2.plot(plot_data.index, plot_data[config], color=f'C{i+2}', label=config)
        
        ax1.set_xlabel('Date')
        ax1.set_ylabel(f'Precipitation & Evaporation [mm/day]')
        ax2.set_ylabel(f'{config} [mm/day]')
        
        ax1.set_title(config)
        
        lines1, labels1 = ax1.get_legend_handles_labels()
        lines2, labels2 = ax2.get_legend_handles_labels()
        ax1.legend(lines1 + lines2, labels1 + labels2, loc='upper left')
        
    plt.tight_layout()
    return fig

# Plot aggregated results
fig = plot_results(results['aggregated'], forcing_data)
plt.show()

## Generate Interactive Maps

Create interactive map visualizations of the results using Folium.

In [ ]:
import folium
import geopandas as gpd
from branca.colormap import LinearColormap
from pathlib import Path
from IPython.display import display

def create_folium_map(gdf, variable, title, background_gdf):
    # Convert to EPSG:4326 for Folium
    gdf_wgs84 = gdf.to_crs(epsg=4326)
    background_gdf_wgs84 = background_gdf.to_crs(epsg=4326)
    
    # Calculate the center point
    center_lat, center_lon = gdf_wgs84.total_bounds[[1, 0]] + (gdf_wgs84.total_bounds[[3, 2]] - gdf_wgs84.total_bounds[[1, 0]]) / 2
    
    # Create a base map with a more toned down style
    m = folium.Map(location=[center_lat, center_lon], zoom_start=11, tiles='CartoDB positron')
    
    # Add background map
    folium.GeoJson(
        background_gdf_wgs84,
        style_function=lambda feature: {
            'fillColor': 'none',
            'color': 'darkgray',
            'weight': 1,
            'fillOpacity': 0,
        }
    ).add_to(m)
    
    # Create a color map
    vmin, vmax = gdf[variable].min(), gdf[variable].max()
    colormap = LinearColormap(colors=['green', 'white'], vmin=vmin, vmax=vmax)
    
    # Add GeoJSON to map
    folium.GeoJson(
        gdf_wgs84,
        style_function=lambda feature: {
            'fillColor': colormap(feature['properties'][variable]),
            'color': 'black',
            'weight': 1,
            'fillOpacity': 0.7,
        },
        tooltip=folium.GeoJsonTooltip(fields=[variable], aliases=[title])
    ).add_to(m)
    
    # Add color map to the map
    colormap.add_to(m)
    colormap.caption = title
    
    return m

# Load geometry data
geo_file = Path(config.input_directory) / Path(config.files.geo_file)
gdf_geometry = gpd.read_file(geo_file)

# Load background shapefile
background_file = Path(config.geodata_directory) / config.files.background_shapefile
background_gdf = gpd.read_file(background_file)

# Ensure both GeoDataFrames use the same CRS
if gdf_geometry.crs != background_gdf.crs:
    background_gdf = background_gdf.to_crs(gdf_geometry.crs)

# Prepare data for mapping (using only the first variable as an example)
variable = 'imported_water'
data = results['local'][variable].groupby(level='cell').sum() / 1000000  # Convert to ML/yr
gdf_geometry[variable] = gdf_geometry['BlockID'].map(data)

# Create and display the map
m = create_folium_map(gdf_geometry, variable, f"{variable.replace('_', ' ').capitalize()} [ML/yr]", background_gdf)
display(m)

# Optionally, save the map to an HTML file
m.save("evapotranspiration_map.html")
print("Map saved to evapotranspiration_map.html")